# **Projeto de Bloco: Ciência de Dados Aplicada**
# **Teste de Performance 2 (TP2)**
## Instituto Infnet - Rafael Dottori de Oliveira

### 23/09/2024

---

### Enunciado

*Na primeira etapa do projeto, você fez a proposta da aplicação, organizou o planejamento inicial e configurou os aspectos básicos do projeto, incluindo a criação de uma aplicação demo em Streamlit.*

*Com isso, você estabeleceu as bases para a solução tecnológica para resolver algum problema ligado aos ODS da Agenda 2030.*

*Agora, é hora de dar um passo adiante e evoluir a sua aplicação, implementando funcionalidades mais avançadas que proporcionarão maior interatividade, usabilidade e eficiência.*

*O TP2 focará na ampliação das capacidades técnicas e na integração de novas funcionalidades, permitindo que sua aplicação não apenas visualize dados, mas também interaja de maneira dinâmica com os usuários e processe informações de fontes externas.*

> *Objetivo:*

*No segundo TP do projeto, você terá a oportunidade de aprimorar e evoluir a aplicação desenvolvida na fase anterior.*

*Nesta etapa, o foco será na configuração do ambiente de desenvolvimento, na implementação de uma interface de usuário mais interativa e na integração de novas funcionalidades para melhorar a experiência do usuário e a performance da aplicação.*

> *Entrega:*

*Ao final desta etapa, sua aplicação deverá estar significativamente mais avançada, com uma interface de usuário interativa e funcional, capacidade de extrair e processar dados de fontes externas, e melhor performance graças ao uso de cache e estado de sessão.*

*Além disso, a documentação do projeto, incluindo o Project Charter e o Data Summary Report, deverá estar completa e refletir o progresso feito até o momento.*

> *Dicas:*

*Lembre-se de testar todas as funcionalidades implementadas para garantir que a aplicação esteja funcionando corretamente.*

*Utilize o controle de versão com Git para documentar e organizar as mudanças no código.*

*Use uma plataforma como Github para armazenar o repositório.*

*Mantenha a organização dos diretórios e arquivos, seguindo as melhores práticas de desenvolvimento.*

---

## **Exercício 1 - Configuração do Ambiente de Desenvolvimento**

*Configure seu ambiente de desenvolvimento, incluindo Git para controle de versão e preparação para deploy. Lembre-se de seguir a estrutura do CRISP-DM para organizar seu projeto de forma eficiente e escalável.*

• **Ambiente Virtual e Instalações**

Iremos reaproveitar o ambiente virtual do TP1, mas poderíamos criar um novo usando o seguinte comando no terminal, certificando que estamos no diretório desejado:

> python -m venv .venv_app

Onde ".venv_app" é o nome do nosso ambiente. Em seguida, ativamos o ambiente, também via terminal:

> .venv_app/Scripts/activate

Criamos um arquivo "requirements.txt" com as bibliotecas que desejamos instalar, como o pandas, streamlit e jupyter. Para realizar essas instalações no ambiente virtual ativado, digitamos:

> python -m pip install -r requirements.txt

• **Git**

Feita a configuração local, podemos criar um novo projeto no GitHub pelo próprio Visual Studio Code.

No menu lateral, temos a aba de Source Control onde podemos publicar o projeto no GitHub, selecionando quais arquivos locais queremos subir para o repositório, como na imagem abaixo:

![create_git](./docs/git_create_project.png)

Uma funcionalidade importante do Git é o *commit* — onde publicamos as mudanças realizadas nos arquivos, por exemplo no arquivo do aplicativo Streamlit. É possível verificar todas as linhas com alterações e subir essas mudanças para o repositório no GitHub.

![commit_changes_git](./docs/git_commit_changes.png)

Além disso, podemos testar alterações em diferentes ramificações (*branches*) do projeto, podemos importar para nossa máquina mudanças feitas por outros contribuidores com o *pull*, baixar versões anteriores, etc.

Essas operações do Git nos permitem controlar a progressão do projeto, acessar diferentes etapas da modelagem, sincronizar o trabalho de uma equipe com diversas pessoas.

• **CRISP-DM**

O controle de versionamento citado acima também está ligado as fases do CRISP-DM ou TDSP, já que essas metodologias podem ser pensadas de maneira cíclica. Ou seja, é possível retomar diferentes etapas do projeto de acordo com nossas necessidades.

Por exemplo, durante a etapa de modelagem ou implementação, talvez seja preciso reavaliar o entendimento de negócio ou dos dados. Podemos então aplicar uma nova abordagem de modelagem em um *branch* diferente. Ou podemos dar continuidade a uma funcionalidade que estava no começo do trabalho mas havia sido deixada de lado.

• **GitHub:** https://github.com/R-Dottori/filme-em-foco

• **Streamlit**: https://filme-em-foco-tp2.streamlit.app/

---

## **Exercício 2 - Implementação de Interface de Usuário Dinâmica**

*Evolua a interface inicial da sua aplicação Streamlit, acrescentando elementos de interatividade que permitam ações dinâmicas por parte do usuário. A interface deve ser intuitiva e funcional, garantindo uma boa experiência de uso.*

Ao longo do trabalho, iremos fazer recortes do aplicativo final (que está no último item desse *notebook*) para **destacar o código e as funcionalidades exigidas em cada exercício**.

Aqui, subimos diretamente a base dos complexos obtidos via a API do Ingresso.com (o processo inteiro está no *notebook* em ./modeling/api_query.ipynb) e permitimos ao usuário filtrar os cinemas por bairro.

Além disso, uma versão inicial do mapa interativo que divide o município do Rio de Janeiro por suas Regiões Administrativas e exibe a localização dos complexos citados acima (esses códigos também estão em ./modeling/folium_map.ipynb).

In [16]:
%%writefile ./apps/tp2_ex2.py

import streamlit as st
import pandas as pd
import geopandas as gpd
import folium
from streamlit_folium import st_folium

st.title('Interface de Usuário')

st.header('Complexos de Cinema no Rio de Janeiro')
salas_rj = pd.read_csv('./data/salas_rj.csv', index_col=0)
bairros = sorted(salas_rj['neighborhood'].unique())
filtro = st.multiselect(label='Selecione quais bairros deseja exibir:', options=bairros, default=bairros)
st.write(salas_rj[['name', 'neighborhood']][salas_rj['neighborhood'].isin(filtro)].sort_values('neighborhood'))

st.header('Mapa dos Complexos por Região Administrativa')
shapefile_rj = gpd.read_file('./data/Regioes Administrativas - RAs - PCRJ.zip')
lat = -22.92
lon = -43.47
mapa_rj = folium.Map(location=[lat, lon], zoom_start=10)
folium.GeoJson(shapefile_rj.to_json(), name='Regiões Administrativas',
                   style_function=lambda feature: {
        'fillColor': '#24b1f2',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5',
        'fillOpacity': 0.5,
    }
).add_to(mapa_rj)

for idx, sala in salas_rj.iterrows():
    folium.Marker(
        location=[sala['latitude'], sala['longitude']],
        tooltip=sala['name'],
        icon=folium.Icon(color='white')
    ).add_to(mapa_rj)

folium.LayerControl().add_to(mapa_rj)
st_folium(mapa_rj)

Overwriting ./apps/tp2_ex2.py


---

## **Exercício 3 - Extração de Conteúdo da Web para alimentar a aplicação**

*Utilize a ferramenta Beautiful Soup para extrair conteúdo de páginas web. Execute esses códigos separadamente e armazene os dados obtidos em arquivos CSV e/ou TXT nos diretórios de data/.
Posteriormente, utilize esses dados para alimentar a interface da aplicação: exiba informações relevantes geradas a partir do conteúdo obtido, como nuvens de palavras e estatísticas básicas (tabelas, notícias).*

Nesse exercício usamos os filtros para não só selecionar um bairro do Rio de Janeiro como escolher um complexo específico.

Depois, mostramos quais os filmes estão em cartaz atualmente no cinema escolhido (com uma nova consulta a API).

Então, com o filme selecionado, fazemos uma raspagem de dados no portal IMDb, retirando o título, nome do diretor e as análises dos usuários.

Com as análises, montamos uma nuvem de palavras destacando os termos mais comuns.

(A etapa de raspagem de dados está em ./modeling/web_scraping.ipynb)

In [72]:
%%writefile ./apps/tp2_ex3.py

import streamlit as st
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Carregando Dados Básicos
salas_rj = pd.read_csv('./data/salas_rj.csv', index_col=0)
UA = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
raiz_api = 'https://api-content.ingresso.com/v0/Sessions/city/2/theater/{}'
filme_escolhido = ''


# Título do Exercício 3
st.title('Raspagem de Dados')

# Escolha do bairro e complexo de cinema
st.header('Escolha um bairro e um cinema:')

bairros = sorted(salas_rj['neighborhood'].unique())
filtro_bairro = st.selectbox(label='Selecione um bairro:', options=bairros)
salas_bairro = salas_rj[salas_rj['neighborhood'] == filtro_bairro]['name'].unique()

filtro_cinema = st.selectbox(label='Selecione um complexo de cinema:', options=salas_bairro)
id_cinema = salas_rj[salas_rj['name'] == filtro_cinema]['id'].values[0]


# Resgatando os filmes em cartaz do complexo escolhido
st.header('Escolha um filme:')

try:
    resp = requests.get(
        url=raiz_api.format(id_cinema),
        headers={'user-agent': UA}
        )

    dados_complexo = resp.json()
    filmes_complexo = []

    for dia in dados_complexo:
        for filme in dia['movies']:
            filmes_complexo.append(filme)

    filmes_cartaz = pd.DataFrame(filmes_complexo)
    st.write(filmes_cartaz[['title', 'originalTitle']].drop_duplicates().sort_values('title'))
    filme_escolhido = st.selectbox(label='Selecione um filme:', options=sorted(filmes_cartaz['title'].drop_duplicates()))
except:
    st.error('ERRO! Não encontramos sessões para o complexo escolhido. Por favor, selecione outro cinema.')

# Após escolher um filme, faremos operações de raspagem no portal IMDb
if filme_escolhido != '':
    st.header('IMDb')

    # Selecionando o primeiro resultado da busca, e esperando que seja igual ao filme escolhido
    url_busca = f'https://www.imdb.com/find/?q={filme_escolhido}&ref_=nv_sr_sm'
    resp_imdb = requests.get(
        url=url_busca,
        headers={'user-agent': UA}
        )
    soup_busca = bs(resp_imdb.text)

    # Raspando a página do primeiro resultado da busca
    url_filme = 'https://www.imdb.com/' + soup_busca.select('ul > li > div > div > a')[0]['href']
    resp_filme = requests.get(
        url=url_filme,
        headers={'user-agent': UA}
    )
    soup_filme = bs(resp_filme.text)

    titulo_filme = soup_filme.select('h1 > span')[0].text
    diretor_filme = soup_filme.select('ul > li > div > ul > li > a')[0].text

    # Entrando nas análises dos usuários pro filme e raspando os textos
    url_analises = 'https://www.imdb.com/' + soup_filme.select('section[data-testid="UserReviews"] > div > div > a')[0]['href']
    resp_analises = requests.get(
                    url=url_analises,
                    headers={'user-agent': UA}
                )   
    soup_analises = bs(resp_analises.text)
    analises = ''
    for tag in soup_analises.select('div[class="lister-list"]'):
        for analise in tag.select('div[class="text show-more__control"]'):
            analises += (analise.text.replace('\n', '').replace("'", "")) + ' '

    # Exibindo um pouco da raspagem, o que também confirma se o primeiro resultado da busca estava correto
    st.subheader('Informações do Filme:')
    st.write(f'Título: {titulo_filme}')
    st.write(f'Diretor: {diretor_filme}')

    # Nuvem de Palavras com as análises
    st.subheader('Nuvem de Palavras')
    nuvem = WordCloud().generate(analises)
    plt.figure()
    plt.imshow(nuvem)
    plt.axis('off')
    st.pyplot(plt)

Overwriting ./apps/tp2_ex3.py


---

## **Exercício 4: Cache e Estado de Sessão**

*Implemente cache e estado de sessão em Streamlit para melhorar a performance da aplicação e garantir a persistência dos dados em aplicações interativas. Isso permitirá que os dados sejam mantidos ao longo das interações do usuário, proporcionando uma experiência mais fluida*

Aqui começamos a explorar um aplicativo com páginas diferentes para testar a persistência dos dados entre diferentes seções do aplicativo.

Também implementos funcionalidades de cache, para garantir o carregamento mais rápido de bases que já foram usadas.

In [62]:
%%writefile ./apps/tp2_ex4.py

import streamlit as st
import pandas as pd

pag_1_title = 'Página 1 - Carregando os Dados'
pag_2_title = 'Página 2 - Exibindo os Dados'


@st.cache_data
def subir_base(arquivo):
    df = pd.read_csv(arquivo, index_col=0)
    return df


def pagina_um():
    if 'df' not in st.session_state:
        st.session_state['df'] = None
    
    st.title('Cache e Estado de Sessão')
    st.header(pag_1_title)

    arquivo = st.file_uploader('Insira um arquivo CSV:', type='csv')
    if arquivo is not None:
        with st.spinner('Carregando...'):
            st.session_state['df'] = subir_base(arquivo)
            st.success('Base carregada com sucesso.')


def pagina_dois():
    st.header(pag_2_title)
    if st.session_state['df'] is not None:
        st.write(st.session_state['df'])
    else:
        st.warning('Aguardando o carregamento da base.')


st.sidebar.title('Navegação')
pagina = st.sidebar.radio(label='Escolha uma página:', options=(pag_1_title, pag_2_title))
if pagina == pag_1_title:
    pagina_um()
else:
    pagina_dois()

Overwriting ./apps/tp2_ex4.py


---

## **Exercício 5: Serviço de Upload e Download de Arquivos**

*Desenvolva um serviço de upload e download de arquivos em Streamlit, permitindo que o usuário adicione mais informações ao sistema através de arquivos CSV. Esses dados devem complementar as informações já exibidas na aplicação, tornando-a mais robusta e informativa.*

Nesse exercício, continuamos a estrutura de várias páginas e permitimos aos usuários subirem dados de novos complexos de cinema.

Utilizamos dois complexos fictícios (./data/salas_rj_adicionar.csv) que propõem a criação de cinemas em Regiões Administrativas que não tinham nenhuma sala, ambos na Zona Oeste do Rio de Janeiro.

É possível baixar a nova tabela gerada, e o mapa interativo exibe os novos complexos com um marcador diferente.

In [78]:
%%writefile ./apps/tp2_ex5.py

import streamlit as st
import pandas as pd
import geopandas as gpd
import folium
from streamlit_folium import st_folium

app_title = 'Subir e Baixar Arquivos'
pag_1_title = 'Carregar os Dados'
pag_2_title = 'Exibir Tabela'
pag_3_title = 'Mapa'
pag_4_title = 'Adicionar Complexos'


@st.cache_data
def subir_base(arquivo):
    df = pd.read_csv(arquivo, index_col=0)
    return df


def pagina_um():
    if 'df' not in st.session_state:
        st.session_state['df'] = None

    if 'novo_df' not in st.session_state:
        st.session_state['novo_df'] = None
    
    st.title(app_title)
    st.header(pag_1_title)

    arquivo = st.file_uploader('Insira um arquivo CSV:', type='csv')
    if arquivo is not None:
        with st.spinner('Carregando...'):
            st.session_state['df'] = subir_base(arquivo)
            st.success('Base carregada com sucesso.')


def pagina_dois():
    st.header(pag_2_title)
    if st.session_state['df'] is not None:
        st.write(st.session_state['df'])
        if st.session_state['novo_df'] is not None:
            st.header('Baixar base com os novos dados:')
            df_download = st.session_state['df'].to_csv()
            st.download_button(
                label='Baixar',
                data=df_download,
                file_name='salas_rj_atualizado.csv'
                )
    else:
        st.warning('Aguardando o carregamento da base.')


def pagina_tres():
    st.header(pag_3_title)
    if st.session_state['df'] is not None:
        shapefile_rj = gpd.read_file('./data/Regioes Administrativas - RAs - PCRJ.zip')
        lat = -22.92
        lon = -43.47
        mapa_rj = folium.Map(location=[lat, lon], zoom_start=10)
        folium.GeoJson(shapefile_rj.to_json(), name='Regiões Administrativas',
                        style_function=lambda feature: {
                'fillColor': '#24b1f2',
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5',
                'fillOpacity': 0.5,
            }
        ).add_to(mapa_rj)

        for idx, sala in st.session_state['df'].iterrows():
            if idx <= 40:
                folium.Marker(
                    location=[sala['latitude'], sala['longitude']],
                    tooltip=sala['name'],
                    icon=folium.Icon(color='white')
                ).add_to(mapa_rj)
            else:
                folium.Marker(
                    location=[sala['latitude'], sala['longitude']],
                    tooltip=sala['name'],
                    icon=folium.Icon(icon='glyphicon glyphicon-film', color='red')
                ).add_to(mapa_rj)

        folium.LayerControl().add_to(mapa_rj)
        st_folium(mapa_rj)
    else:
        st.warning('Aguardando o carregamento da base.')


def pagina_quatro():
    st.header(pag_4_title)
    if st.session_state['df'] is not None:
        dados_exemplo = {'id': '1234', 'name': 'Nome do Complexo', 'neighborhood': 'Bairro', 'latitude': -22.92, 'longitude': -43.47}
        df_exemplo = pd.DataFrame(dados_exemplo, index=[0])
        st.write('Para adicionar novos dados, insira um arquivo CSV respeitando o seguinte esquema:')
        st.write(df_exemplo)

        novo_arquivo = st.file_uploader('Insira o arquivo CSV:', type='csv')
        if novo_arquivo is not None:
            with st.spinner('Carregando...'):
                st.session_state['novo_df'] = subir_base(novo_arquivo)
            if set(st.session_state['df'].columns) == set(st.session_state['novo_df'].columns):
                st.write(st.session_state['novo_df'])
                st.session_state['df'] = pd.concat([st.session_state['df'], st.session_state['novo_df']], ignore_index=True)
                st.success('Novos dados adicionados com sucesso.')
                st.header('Baixar base com os novos dados:')
                df_download = st.session_state['df'].to_csv()
                st.download_button(
                    label='Baixar',
                    data=df_download,
                    file_name='salas_rj_atualizado.csv'
                    )
            else:
                st.error('ERRO! Os dados inseridos não seguem o esquema exigido.')
    else:
        st.warning('Aguardando o carregamento da base.')


st.sidebar.title('Navegação')
pagina = st.sidebar.radio(label='Escolha uma página:', options=(pag_1_title, pag_2_title, pag_3_title, pag_4_title))
if pagina == pag_1_title:
    pagina_um()
elif pagina == pag_2_title:
    pagina_dois()
elif pagina == pag_3_title:
    pagina_tres()
else:
    pagina_quatro()

Overwriting ./apps/tp2_ex5.py


---

## **Exercício 6: Finalização do Project Charter e Data Summary Report**

*Complete o Project Charter e o Data Summary Report, detalhando o escopo, os objetivos, os stakeholders do projeto, e as fontes de dados utilizadas.*

#### **Project Charter - Business Model Canvas**

![folders](docs/business_model_canvas.png)

#### **Data Summary Report**

• API do Ingresso.com:

— Fonte: Consultas na API do Ingresso.com

— Formato: Dados textuais em tabelas .CSV

— Objetivo: Catálogo dos complexos de cinema no município do Rio de Janeiro, utilizando uma fonte via API.

— — — 

• Shapefile com os polígonos das Regiões Administrativas do Rio de Janeiro:

— Fonte: Disponibilizado pelo Sistema Municipal de Informações Urbanas- SIURB/PCRJ

— Formato: Arquivo .zip contendo o Shapefile (incluindo outros arquivos .shp, .dbf, .shx, etc.)

— Objetivo: Criar um mapa interativo com as divisões de RAs e mapear cada complexo de cinema de acordo

— — —

• Informações do IMDb:

— Fonte: Raspagem de dados de filmes no portal IMDb

— Formato: Dados textuais contendo informações como título do filme, diretor e análises dos usuários

— Objetivo: Exibir informações dos filmes que não estão disponíveis na API e criar nuvem de palavras com os termos mais usados pelos espectadores

---

## **Aplicativo**

Finalmente temos o **aplicativo final do TP2**, incluindo todas as funcionalidades exigidas nos exercícios.

**GitHub:** https://github.com/R-Dottori/filme-em-foco

**Streamlit**: https://filme-em-foco-tp2.streamlit.app/

In [93]:
%%writefile ./apps/tp2_app_final.py

import streamlit as st
import pandas as pd
import geopandas as gpd
import folium
from streamlit_folium import st_folium
import requests
from bs4 import BeautifulSoup as bs
from wordcloud import WordCloud
import matplotlib.pyplot as plt


app_title = 'Subir e Baixar Arquivos'
pag_1_title = 'Carregar os Dados'
pag_2_title = 'Exibir Tabela'
pag_3_title = 'Mapa'
pag_4_title = 'Adicionar Complexos'
pag_5_title = 'Escolher um Filme'


@st.cache_data
def subir_base(arquivo):
    df = pd.read_csv(arquivo, index_col=0)
    return df


def pagina_um():
    if 'df' not in st.session_state:
        st.session_state['df'] = None

    if 'novo_df' not in st.session_state:
        st.session_state['novo_df'] = None
    
    st.title(app_title)
    st.header(pag_1_title)

    arquivo = st.file_uploader('Insira um arquivo CSV:', type='csv')
    if arquivo is not None:
        with st.spinner('Carregando...'):
            st.session_state['df'] = subir_base(arquivo)
            st.success('Base carregada com sucesso.')


def pagina_dois():
    st.header(pag_2_title)
    if st.session_state['df'] is not None:
        st.write(st.session_state['df'])
        if st.session_state['novo_df'] is not None:
            st.header('Baixar base com os novos dados:')
            df_download = st.session_state['df'].to_csv()
            st.download_button(
                label='Baixar',
                data=df_download,
                file_name='salas_rj_atualizado.csv'
                )
    else:
        st.warning('Aguardando o carregamento da base.')


def pagina_tres():
    st.header(pag_3_title)
    if st.session_state['df'] is not None:
        shapefile_rj = gpd.read_file('./data/Regioes Administrativas - RAs - PCRJ.zip')
        lat = -22.92
        lon = -43.47
        mapa_rj = folium.Map(location=[lat, lon], zoom_start=10)
        folium.GeoJson(shapefile_rj.to_json(), name='Regiões Administrativas',
                        style_function=lambda feature: {
                'fillColor': '#24b1f2',
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5',
                'fillOpacity': 0.5,
            }
        ).add_to(mapa_rj)

        for idx, sala in st.session_state['df'].iterrows():
            if idx <= 40:
                folium.Marker(
                    location=[sala['latitude'], sala['longitude']],
                    tooltip=sala['name'],
                    icon=folium.Icon(color='white')
                ).add_to(mapa_rj)
            else:
                folium.Marker(
                    location=[sala['latitude'], sala['longitude']],
                    tooltip=sala['name'],
                    icon=folium.Icon(icon='glyphicon glyphicon-film', color='red')
                ).add_to(mapa_rj)

        folium.LayerControl().add_to(mapa_rj)
        st_folium(mapa_rj)
    else:
        st.warning('Aguardando o carregamento da base.')


def pagina_quatro():
    st.header(pag_4_title)
    if st.session_state['df'] is not None:
        dados_exemplo = {'id': '1234', 'name': 'Nome do Complexo', 'neighborhood': 'Bairro', 'latitude': -22.92, 'longitude': -43.47}
        df_exemplo = pd.DataFrame(dados_exemplo, index=[0])
        st.write('Para adicionar novos dados, insira um arquivo CSV respeitando o seguinte esquema:')
        st.write(df_exemplo)

        novo_arquivo = st.file_uploader('Insira o arquivo CSV:', type='csv')
        if novo_arquivo is not None:
            with st.spinner('Carregando...'):
                st.session_state['novo_df'] = subir_base(novo_arquivo)
            if set(st.session_state['df'].columns) == set(st.session_state['novo_df'].columns):
                st.write(st.session_state['novo_df'])
                st.session_state['df'] = pd.concat([st.session_state['df'], st.session_state['novo_df']], ignore_index=True)
                st.success('Novos dados adicionados com sucesso.')
                st.header('Baixar base com os novos dados:')
                df_download = st.session_state['df'].to_csv()
                st.download_button(
                    label='Baixar',
                    data=df_download,
                    file_name='salas_rj_atualizado.csv'
                    )
            else:
                st.error('ERRO! Os dados inseridos não seguem o esquema exigido.')
    else:
        st.warning('Aguardando o carregamento da base.')


def pagina_cinco():
    st.header(pag_5_title)
    UA = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
    raiz_api = 'https://api-content.ingresso.com/v0/Sessions/city/2/theater/{}'
    filme_escolhido = ''
    if st.session_state['df'] is not None:
        # Escolha do bairro e complexo de cinema
        st.header('Escolha um bairro e um cinema:')

        bairros = sorted(st.session_state['df']['neighborhood'].unique())
        filtro_bairro = st.selectbox(label='Selecione um bairro:', options=bairros)
        salas_bairro = st.session_state['df'][st.session_state['df']['neighborhood'] == filtro_bairro]['name'].unique()

        filtro_cinema = st.selectbox(label='Selecione um complexo de cinema:', options=salas_bairro)
        id_cinema = st.session_state['df'][st.session_state['df']['name'] == filtro_cinema]['id'].values[0]


        # Resgatando os filmes em cartaz do complexo escolhido
        st.header('Escolha um filme:')

        try:
            resp = requests.get(
                url=raiz_api.format(id_cinema),
                headers={'user-agent': UA}
                )

            dados_complexo = resp.json()
            filmes_complexo = []

            for dia in dados_complexo:
                for filme in dia['movies']:
                    filmes_complexo.append(filme)

            filmes_cartaz = pd.DataFrame(filmes_complexo)
            st.write(filmes_cartaz[['originalTitle']].drop_duplicates().sort_values('originalTitle'))
            filme_escolhido = st.selectbox(label='Selecione um filme:', options=sorted(filmes_cartaz['originalTitle'].drop_duplicates()))
        except:
            st.error('ERRO! Não encontramos sessões para o complexo escolhido. Por favor, selecione outro cinema.')

        # Após escolher um filme, faremos operações de raspagem no portal IMDb
        if filme_escolhido != '':
            st.header('IMDb')
            try:
                with st.spinner('Carregando...'):
                    # Selecionando o primeiro resultado da busca, e esperando que seja igual ao filme escolhido
                    url_busca = f'https://www.imdb.com/find/?q={filme_escolhido}&ref_=nv_sr_sm'
                    resp_imdb = requests.get(
                        url=url_busca,
                        headers={'user-agent': UA}
                        )
                    soup_busca = bs(resp_imdb.text)

                    # Raspando a página do primeiro resultado da busca
                    url_filme = 'https://www.imdb.com/' + soup_busca.select('ul > li > div > div > a')[0]['href']
                    resp_filme = requests.get(
                        url=url_filme,
                        headers={'user-agent': UA}
                    )
                    soup_filme = bs(resp_filme.text)

                    titulo_filme = soup_filme.select('h1 > span')[0].text
                    diretor_filme = soup_filme.select('ul > li > div > ul > li > a')[0].text

                    # Entrando nas análises dos usuários pro filme e raspando os textos
                    url_analises = 'https://www.imdb.com/' + soup_filme.select('section[data-testid="UserReviews"] > div > div > a')[0]['href']
                    resp_analises = requests.get(
                                    url=url_analises,
                                    headers={'user-agent': UA}
                                )   
                    soup_analises = bs(resp_analises.text)
                    analises = ''
                    for tag in soup_analises.select('div[class="lister-list"]'):
                        for analise in tag.select('div[class="text show-more__control"]'):
                            analises += (analise.text.replace('\n', '').replace("'", "")) + ' '

                    # Exibindo um pouco da raspagem, o que também confirma se o primeiro resultado da busca estava correto
                    st.subheader('Informações do Filme:')
                    st.write(f'Título: {titulo_filme}')
                    st.write(f'Diretor: {diretor_filme}')

                    # Nuvem de Palavras com as análises
                    st.subheader('Nuvem de Palavras')
                    nuvem = WordCloud().generate(analises)
                    plt.figure()
                    plt.imshow(nuvem)
                    plt.axis('off')
                    st.pyplot(plt)
            except:
                st.error('ERRO! Filme não encontrado no IMDb, por favor selecione outro.')
    else:
        st.warning('Aguardando o carregamento da base.')


st.sidebar.title('Navegação')
pagina = st.sidebar.radio(label='Escolha uma página:', options=(pag_1_title, pag_2_title, pag_3_title, pag_4_title, pag_5_title))
if pagina == pag_1_title:
    pagina_um()
elif pagina == pag_2_title:
    pagina_dois()
elif pagina == pag_3_title:
    pagina_tres()
elif pagina == pag_4_title:
    pagina_quatro()
else:
    pagina_cinco()

Overwriting ./apps/tp2_app_final.py
